# Generator
* Feature 
    * Architecture : Any
    * 역변환을 통해서 실제값 만들기

# Discriminator
* Feature 
    * Architecture : TABNET
    * 전처리 되지 않은 데이터를 바로 사용하기

In [1]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import spectral_norm as SN
import torch
from IPython.display import clear_output
from sklearn.preprocessing import LabelEncoder

import os
from argparse import ArgumentParser
from collections import OrderedDict

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST

import pytorch_lightning as pl
cuda = True if torch.cuda.is_available() else False

In [2]:
sys.path.append("TABNET_simple")
from tab_network import TabNet

In [3]:
data = pd.read_csv("./dataset/only_cat/train.csv")
data.head()


,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [4]:
data.drop(["id", "target"],axis=1, inplace= True)

In [5]:
df = data.filter(regex="bin|month")

In [6]:
fac_cols = df.columns.tolist()
num_cols = []
df[fac_cols] = df[fac_cols].astype("object")


/opt/conda/envs/test/lib/python3.8/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [7]:
types = df.dtypes

categorical_columns = []
categorical_dims =  {}
for col in fac_cols  + num_cols :
    if types[col] == 'object' :
        l_enc = LabelEncoder()
        df[col] = df[col].fillna("VV_likely")
        df[col] = l_enc.fit_transform(df[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        df.fillna(df.loc[:, col].mean(), inplace=True)


<ipython-input-7-64552185dea0>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna("VV_likely")
<ipython-input-7-64552185dea0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = l_enc.fit_transform(df[col].values)


In [8]:
features = [ col for col in df.columns if col in fac_cols + num_cols] 
cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]


In [9]:
input_dim = len(features)
output_dim = 1
from typing import List, Any, Dict
import torch
n_d: int = 5
n_a: int = 5
n_steps: int = 5
gamma: float = 1.3
cat_idxs: List[int] =[]
cat_dims: List[int] =[]
cat_emb_dim: int = 3
n_independent: int = 2
n_shared: int = 3
epsilon: float = 1e-15
momentum: float = 0.02
lambda_sparse: float = 1e-3
seed: int = 2345
clip_value: int = 1
verbose: int = 1
optimizer_fn: Any = torch.optim.Adam
optimizer_params: Dict = dict(lr=2e-2)
scheduler_fn: Any = None
scheduler_params: Dict = {"step_size":50,"gamma":0.9}
mask_type: str = "entmax" # "sparsemax"
input_dim: int = input_dim
output_dim: int = output_dim
device_name: str = "auto"
virtual_batch_size=128


In [10]:
import category_encoders as ce 
df.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,month
0,0,0,0,1,1,1
1,0,1,0,1,1,7
2,0,0,0,0,1,1
3,0,1,0,0,1,0
4,0,0,0,0,0,7


In [11]:
encoder = ce.OneHotEncoder(cols=fac_cols, use_cat_names= True ,handle_unknown="ignore",)
encoder.fit(df)
df_trans = encoder.transform(df)
encoder.inverse_transform(df_trans)

/opt/conda/envs/test/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,bin_0,bin_1,bin_2,bin_3,bin_4,month
0,0,0,0,1,1,1
1,0,1,0,1,1,7
2,0,0,0,0,1,1
3,0,1,0,0,1,0
4,0,0,0,0,0,7
...,...,...,...,...,...,...
299995,0,0,0,1,0,7
299996,0,0,0,0,1,1
299997,0,0,0,0,1,8
299998,0,1,0,0,1,7


In [16]:
df_trans

from sklearn.preprocessing import MinMaxScaler
import umap

In [17]:
reducer = umap.UMAP(random_state=2345)
reducer.fit(df_trans.values)

KeyboardInterrupt: 

In [ ]:
embedding = reducer.transform(df_trans.values)
plt.scatter(embedding[:, 0], embedding[:, 1])
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the Digits dataset', fontsize=24)

In [89]:
from copy import deepcopy
map_info = deepcopy(encoder.category_mapping)
[_dict_.update({"len" : len(_dict_["mapping"])-1}) for _dict_ in map_info]

[None, None, None, None, None, None]

In [90]:
from torch.nn.functional import gumbel_softmax as g_softmax

In [91]:
def logit2onehot(logit, map_info) :
    cur_position = 0
    result = []
    for col_info in map_info :
        cur_position += col_info["len"]
        tt = logit[:, (cur_position-col_info["len"]):cur_position]
        onehot_ = g_softmax(logits= tt , hard=True,tau=1)
        result.append(onehot_)
    return torch.cat(result,axis=1)

from torch.distributions import Categorical
def logit2dist(logit,map_info) :
    cur_position = 0
    result = []
    for col_info in map_info :
        cur_position += col_info["len"]
        tt = logit[:, (cur_position-col_info["len"]):cur_position]
        dist = Categorical(logits = tt)
        result.append(dist)
    return result


In [92]:
total_column = df_trans.columns.tolist()
total_column


['bin_0_0.0',
 'bin_0_1.0',
 'bin_1_0.0',
 'bin_1_1.0',
 'bin_2_0.0',
 'bin_2_1.0',
 'bin_3_1.0',
 'bin_3_0.0',
 'bin_4_1.0',
 'bin_4_0.0',
 'month_1.0',
 'month_7.0',
 'month_0.0',
 'month_3.0',
 'month_9.0',
 'month_2.0',
 'month_6.0',
 'month_8.0',
 'month_11.0',
 'month_10.0',
 'month_4.0',
 'month_5.0']

In [93]:
# from dataclasses import dataclass, field
# @dataclass
# class Book :
#     n_d: int
#     n_a: int = 8
#     n_steps: int = 3

#     def __post_init__(self,data) :
#         self.full = self.n_d + self.n_a
# Book()

In [94]:
class TabularDataModule(pl.LightningDataModule) :
    def __init__(self, data , batch_size:int = 32 , num_workers:int=3) :
        super().__init__()
        self.data = data
        self.batch_size = batch_size
        self.num_workers = num_workers

        self.dims = self.data.shape[1]
    
    def prepare_data(self,) :
        pass

    def setup(self,stage=None) :
        if stage == "fit" or state is None :
            train_length = int(len(self.data)*0.8)
            lengths = [train_length, int(len(self.data)-train_length)]
            self.train, self.val = random_split(self.data, lengths)
        
        if stage == "test" or stage is None :
            self.test = self.data   


    def train_dataloader(self):
        return DataLoader(self.train , batch_size= self.batch_size , num_workers=self.num_workers )

    def valid_dataloader(self):
        return DataLoader(self.val , batch_size= self.batch_size , num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test , batch_size= self.batch_size , num_workers=self.num_workers)



In [95]:
class Generator(nn.Module):
    def __init__(self, latent_dim, out_shape, map_info , encoder, total_column):
        super().__init__()
        self.out_shape = out_shape
        self.map_info = map_info
        self.encoder = encoder
        self.total_column = total_column
        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=True),
            *block(128, 128, normalize=True),
            *block(128, 64, normalize=True),
            *block(64, 32, normalize=True),
            nn.Linear(32, self.out_shape),
        )

    def forward(self, z):
        x = self.model(z)
        # img = img.view(img.size(0), *self.out_shape)
        return x

    def inference(self,z) :
        x = self.model(z)
        x = self.logit2onehot(x)
        return self.onehot2rev(x)

    def inference_tabnet_training(self,z):
        return torch.FloatTensor(self.inference(z).values)

    def splitbycolumn(self,logit) :
        cur_position = 0
        result = []
        for col_info in self.map_info :
            cur_position += col_info["len"]
            tt = logit[:, (cur_position-col_info["len"]):cur_position]
            result.append(tt)
        return result

    def logit2index(self,logit) :
        cur_position = 0
        result = []
        for col_info in self.map_info :
            cur_position += col_info["len"]
            tt = logit[:, (cur_position-col_info["len"]):cur_position]
            onehot_ = g_softmax(logits= tt , hard=True,tau=1)
            result.append(torch.argmax(tt,axis=1))
        return result

    def logit2onehot(self,logit) :
        cur_position = 0
        result = []
        for col_info in self.map_info :
            cur_position += col_info["len"]
            tt = logit[:, (cur_position-col_info["len"]):cur_position]
            onehot_ = g_softmax(logits= tt , hard=True,tau=1)
            result.append(onehot_)
        return torch.cat(result,axis=1)
    
    def logit2dist(self,logit) :
        cur_position = 0
        result = []
        for col_info in self.map_info :
            cur_position += col_info["len"]
            tt = logit[:, (cur_position-col_info["len"]):cur_position]
            dist = Categorical(logits = tt)
            result.append(dist)
        return result

    def onehot2rev(self,tensor) :
        gene_onehot_pd = pd.DataFrame(tensor.detach().numpy(),
        columns=self.total_column)
        gene_data = self.encoder.inverse_transform(gene_onehot_pd)
        return gene_data
    


In [96]:
generator = Generator(latent_dim=30, out_shape=len(total_column),
map_info=map_info,encoder=encoder , total_column=total_column)




In [97]:
z = torch.randn(100, 30)
gene_logit = generator(z)
gene_onehot_tensor = logit2onehot(gene_logit, map_info)
gene_onehot_pd = pd.DataFrame(gene_onehot_tensor.detach().numpy(),columns=total_column)
gene_data = encoder.inverse_transform(gene_onehot_pd)
gene_data

,bin_0,bin_1,bin_2,bin_3,bin_4,month
0,1,0,1,1,1,9
1,0,1,1,1,1,6
2,1,0,1,0,1,10
3,0,1,1,0,1,10
4,0,0,0,0,1,0
...,...,...,...,...,...,...
95,0,1,1,1,0,0
96,1,1,0,0,1,8
97,1,0,1,0,0,2
98,0,1,1,0,1,1


In [98]:
z = torch.randn(100, 30)
# generator.inference(z)
# generator.splitbycolumn(z)

In [99]:
[dist.entropy().mean() for dist in logit2dist(gene_logit , map_info)]

[tensor(0.6880, grad_fn=<MeanBackward0>),
 tensor(0.6881, grad_fn=<MeanBackward0>),
 tensor(0.6929, grad_fn=<MeanBackward0>),
 tensor(0.6881, grad_fn=<MeanBackward0>),
 tensor(0.6927, grad_fn=<MeanBackward0>),
 tensor(2.4807, grad_fn=<MeanBackward0>)]

In [100]:
from functools import partial
Discriminator = partial(TabNet, 
            output_dim = 1,
            n_d=n_d,
            n_a=n_a,
            n_steps=n_steps,
            gamma=gamma,
            cat_idxs=cat_idxs,
            cat_dims=cat_dims,
            cat_emb_dim=cat_emb_dim,
            n_independent=n_independent,
            n_shared=n_shared,
            epsilon=epsilon,
            virtual_batch_size=virtual_batch_size,
            momentum=momentum,
            mask_type=mask_type
)

In [101]:
# disc = Discriminator(dm.size())
# disc(torch.FloatTensor(generator.inference(z).values))


In [102]:
class GAN(pl.LightningModule): 
    def __init__(self,
                gene_output_dim = None,
                disc_input_dim = None,
                latent_dim =100 , 
                lr: float = 0.0002,
                b1: float = 0.5,
                b2: float = 0.999,
                generator_parm:dict={},
                discriminator_parm:dict={},
                # batch_size: int = 64,
                **kwargs) :
        super().__init__()
        self.save_hyperparameters()

        self.generator = Generator(latent_dim=self.hparams.latent_dim,
        out_shape=self.hparams.gene_output_dim,**generator_parm)

        self.discriminator = Discriminator(input_dim=self.hparams.disc_input_dim,**discriminator_parm)

        self.validation_z = torch.randn(8, self.hparams.latent_dim)

        self.example_input_array = torch.zeros(2, self.hparams.latent_dim)

    def forward(self, z):
        return self.generator(z)
    
    def inference(self,z) :
        return self.generator.inference(z)

    def adversarial_loss(self, y_hat, y):
        return F.binary_cross_entropy(y_hat, y)
    
    def validation_step(self, batch, batch_idx):
        x = batch
        z = torch.randn(x.shape[0], self.hparams.latent_dim)
        z = z.type_as(x)
        generated_x = self.generator.inference_tabnet_training(z)
        valid = torch.ones(x.size(0), 1)
        valid = valid.type_as(x)
        logit,M_loss = self.discriminator(generated_x)
        reward = F.binary_cross_entropy(nn.Sigmoid()(logit), valid,reduction ='mean').detach()
        self.log('val_reward', reward)

    def training_step(self, batch, batch_idx, optimizer_idx):
        x = batch
        # sample noise
        z = torch.randn(x.shape[0], self.hparams.latent_dim)
        z = z.type_as(x)
        # train generator
        if optimizer_idx == 0:
            ## TODO: how to training generator? 
            ## TODO: 현재 강화학습 방식을 도입하여 유사한 로스를 설계해서 진행중...(어디에도 없는 방식이라 확신 없음)
            
            # generate images
            self.generated_imgs = self.generator.inference_tabnet_training(z)

            # ground truth result (ie: all fake)
            # put on GPU because we created this tensor inside training_loop
            valid = torch.ones(x.size(0), 1)
            valid = valid.type_as(x)

            # adversarial loss is binary cross-entropy
            logit,M_loss = self.discriminator(self.generated_imgs)
            # g_loss = self.adversarial_loss(nn.Sigmoid()(logit), valid).detach()
            reward = 1/F.binary_cross_entropy(nn.Sigmoid()(logit), valid,reduction ='none').detach()
            logit = self.generator(z)
            list_of_dist = self.generator.logit2dist(logit)
            list_of_action = self.generator.logit2index(logit)
            g_loss_list = [(dist.log_prob(action) * reward.squeeze()).mean() for dist , action in zip(list_of_dist,list_of_action)]
            # print(g_loss)
            g_loss = torch.stack(g_loss_list).mean()
            tqdm_dict = {'g_loss': g_loss}
            output = OrderedDict({
                'loss': g_loss,
                'progress_bar': tqdm_dict,
                'log': tqdm_dict
            })
            return output

        # train discriminator
        if optimizer_idx == 1:
            # Measure discriminator's ability to classify real from generated samples

            # how well can it label as real?
            valid = torch.ones(x.size(0), 1)
            valid = valid.type_as(x)
            logit,M_loss = self.discriminator(x)
            real_loss = self.adversarial_loss(nn.Sigmoid()(logit), valid)

            # how well can it label as fake?
            fake = torch.zeros(x.size(0), 1)
            fake = fake.type_as(x)
            logit,M_loss =self.discriminator(self.generator.inference_tabnet_training(z).detach())
            fake_loss = self.adversarial_loss(nn.Sigmoid()(logit) , fake)

            # discriminator loss is the average of these
            d_loss = (real_loss + fake_loss) / 2
            tqdm_dict = {'d_loss': d_loss}
            output = OrderedDict({
                'loss': d_loss,
                'progress_bar': tqdm_dict,
                'log': tqdm_dict
            })
            return output

    def configure_optimizers(self):
        lr = self.hparams.lr
        b1 = self.hparams.b1
        b2 = self.hparams.b2
        opt_g = torch.optim.Adam(self.generator.parameters(), lr=lr, betas=(b1, b2))
        opt_d = torch.optim.Adam(self.discriminator.parameters(), lr=lr, betas=(b1, b2))
        return [opt_g, opt_d], []

    def on_epoch_end(self):
        z = self.validation_z.type_as(self.generator.model[0].weight)


In [103]:
generator_parm = dict(map_info=map_info,encoder=encoder,total_column = total_column)

In [104]:
dm = TabularDataModule(df.values.astype(np.float32),batch_size=1024)
model = GAN(gene_output_dim = len(total_column), disc_input_dim = dm.size(), generator_parm = generator_parm)

In [105]:

trainer = pl.Trainer(gpus=0, max_epochs=10, progress_bar_refresh_rate=20)
trainer.fit(model, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/opt/conda/envs/test/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: you defined a validation_step but have no val_dataloader. Skipping val loop
  warnings.warn(*args, **kwargs)

  | Name          | Type      | Params | In sizes | Out sizes
-------------------------------------------------------------------
0 | generator     | Generator | 41.2 K | [2, 100] | [2, 22]  
1 | discriminator | TabNet    | 4.4 K  | ?        | ?        
-------------------------------------------------------------------
45.6 K    Trainable params
0         Non-trainable params
45.6 K    Total params
0.182     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [ ]:
trainer.save_checkpoint("./cat_gan_model.ckpt")
new_model = model.load_from_checkpoint(checkpoint_path="./cat_gan_model.ckpt")
trainer = pl.Trainer(gpus=0, max_epochs=100, progress_bar_refresh_rate=20)
trainer.fit(new_model, dm)

In [32]:
new_model = model.load_from_checkpoint(checkpoint_path="./cat_gan_model.ckpt")
from pytorch_lightning.callbacks import ModelCheckpoint , EarlyStopping
checkpoint_callback = ModelCheckpoint(
    monitor='val_reward',
    dirpath='./model',
    verbose=True,
    filename='gan-{epoch:02d}-{val_reward:.2f}',
    save_top_k=1, # 3,
    # save_last=True,
    mode='min',
)
early_stopping = EarlyStopping(
    monitor='val_reward',
    patience=100,
    verbose=True,
    mode='min'
)

trainer = pl.Trainer(gpus=0, max_epochs=100, progress_bar_refresh_rate=20,callbacks=[checkpoint_callback,early_stopping])
trainer.fit(new_model, dm)

/opt/conda/envs/test/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Checkpoint directory ./model exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/opt/conda/envs/test/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: you defined a validation_step but have no val_dataloader. Skipping val loop
  warnings.warn(*args, **kwargs)

  | Name          | Type      | Params | In sizes | Out sizes
-------------------------------------------------------------------
0 | generator     | Generator | 41.2 K | [2, 100] | [2, 22]  
1 | discriminator | TabNet    | 4.4 K  | ?        | ?        
-------------------------------------------------------------------
45.6 K    Trainable params
0         Non-trainable params
45.6 K    Total params
0.182     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Epoch 0, global step 234: val_reward was not in top 1
Epoch 1, global step 469: val_reward was not in top 1
/opt/conda/envs/test/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


In [40]:
checkpoint_callback.best_model_path

''

In [46]:
import matplotlib.pyplot as plt
model.eval()
z = torch.randn(10, model.hparams.latent_dim)
model.inference(z)

,bin_0,bin_1,bin_2,bin_3,bin_4,month
0,0,0,1,F,Y,9
1,0,0,1,F,N,7
2,1,0,0,F,Y,8
3,0,0,1,F,Y,9
4,0,0,1,T,N,9
5,1,0,0,F,Y,2
6,0,0,0,T,Y,1
7,0,0,1,F,Y,8
8,1,0,1,T,Y,8
9,0,0,1,T,N,5
